In [5]:
import sys 
if '..' not in sys.path:
    sys.path.append('../')
from utils import *
data_dict = load_data(verbose=True)

CONTENTS OF HDF5 FILE:
pose
	 ekf_ori, tango_ori, tango_pos
raw
	 imu
		 acce, game_rv, gps, gravity, gyro, gyro_uncalib, linacce, magnet, magnetic_rv, pressure, rv, step, wifi_address, wifi_values
	 tango
		 acce, game_rv, gps, gravity, gyro, gyro_uncalib, linacce, magnet, magnetic_rv, pressure, rv, step, tango_adf_pose, tango_pose, wifi_address, wifi_values
synced
	 acce, game_rv, grav, gyro, gyro_uncalib, linacce, magnet, rv, time


In [6]:
w = data_dict['synced/gyro']
a = data_dict['synced/acce']
m = data_dict['synced/magnet']

In [7]:

EAST = np.cross(m[0],a[0])
EAST/=np.linalg.norm(EAST)
DOWN = a[0]                                          # maybe a minus sign is needed
DOWN/=np.linalg.norm(DOWN)
NORTH = np.cross(EAST,DOWN)
NORTH/=np.linalg.norm(NORTH)

# body frame to world frame
R = np.array([NORTH, EAST, DOWN]).T
R, NORTH, EAST, DOWN


(array([[ 0.0638138 ,  0.01855377, -0.99778933],
        [ 0.88993506, -0.45350301,  0.04848313],
        [-0.45160092, -0.8910616 , -0.0454514 ]]),
 array([ 0.0638138 ,  0.88993506, -0.45160092]),
 array([ 0.01855377, -0.45350301, -0.8910616 ]),
 array([-0.99778933,  0.04848313, -0.0454514 ]))

In [8]:
def rotation_matrix_2_quaternion(R):                 # they may not be in the right order
    # R is a 3x3 rotation matrix
    # q is a 4x1 quaternion
    q = np.zeros((4,1))
    q[0] = np.sqrt(1 + R[0,0] + R[1,1] + R[2,2])/2
    q[1] = (R[2,1] - R[1,2])/(4*q[0])
    q[2] = (R[0,2] - R[2,0])/(4*q[0])
    q[3] = (R[1,0] - R[0,1])/(4*q[0])
    return q

rotation_matrix_2_quaternion(np.linalg.inv(R))

array([[ 0.37578564],
       [ 0.62505364],
       [ 0.36336434],
       [-0.57970635]])

In [9]:
# check if this lines up with the quaternion from the Tango
data_dict['pose/tango_ori'][0]

array([ 0.34436653,  0.59040797,  0.35096438, -0.64004271])